In [2]:
'''
Process and save out 10 human plans per goal
'''

import os
import numpy as np 
import pandas as pd 
import random

random.seed(10)


In [5]:
# load in human generations 
# for now, read directly from .txt files
exp_results_dir = "/Users/kcollins/language_and_structure_of_thoughts/plans/exp_results/"
save_dir = exp_results_dir
data_file = "plans_per_goal.txt"

with open(f"{exp_results_dir}unconstrained_planning_pilot/{data_file}", "r") as f: 
    unconstrained_data = f.readlines()
    
with open(f"{exp_results_dir}constrained_single_objs/{data_file}", "r") as f: 
    constrained_single_data = f.readlines()

with open(f"{exp_results_dir}constrained_many_objs/{data_file}", "r") as f: 
    constrained_many_data = f.readlines()



In [12]:
def is_goal(line): 
    # check if a line is a goal
    return line[:4] == "Goal"

def is_plan(line):
    # check if a line is the start of a plan
    return line[1:5] == "Plan"

def process_plan(plan):
    # remove starting "Plan: " and end "/n" 
    plan = plan[7:-1]
    # make sure that plan ends in a period 
    # remove all intermediate breaks -- replace with a period, or nothing if already exists a period
    plan = plan.replace(".<br />", ". ") # keep period
    plan = plan.replace(". <br />", ". ")
    plan = plan.replace("<br />", ". ") # add period 
    
    # for the final quote, make sure that there are not big gaps with extra spaces
    plan = plan.replace(".  \"", ".\"")
    if plan[-3:] == ". \"": 
        plan = plan[:-3] + ".\"" # remove trailing space before end
        
    # if no period at the end of the sentence, add it in
    # (for consistency w/ end token used w/ gpt-3)
    if plan[-2:] != ".\"": plan = plan[:-1] + ".\""
    
    return plan 
    
def get_plans_per_goal(data, goal_type): 
    plans_per_goal = {}
    
    goals = []
    all_plans = []
    goal_types = []
    ids = [] 
    for i, line in enumerate(data): 
        if is_goal(line): 
            goal = line[:-1] # remove ending new line character
            # extract all plans for that goal
            plans = []
            for j, poss_plan in enumerate(data[i+1:]):
                if is_goal(poss_plan): break # on to a new goal --- save prior plans and move-on
                else: 
                    if is_plan(poss_plan): 
                        plan = process_plan(poss_plan)
                        # note: manually inspect that they all are nested in quotes! 
                        formatted_plan_str = plan

                        plans.append(formatted_plan_str)
            plans_per_goal[goal] = {"plans": plans, "n_plans": len(plans)}
            
            # add to lists, along w/ metadata
            all_plans.extend(plans)
            goals.extend([goal for _ in range(len(plans))])
            goal_types.extend([goal_type for _ in range(len(plans))])
            ids.extend(["human" for _ in range(len(plans))])
            
    goal_plan_df = pd.DataFrame({"goal": goals, "plan": all_plans, "id": ids,"goal_type": goal_types})
    return goal_plan_df

dfs = [get_plans_per_goal(unconstrained_data, "unconstrained"), 
      get_plans_per_goal(constrained_single_data, "constrained_single"),
      get_plans_per_goal(constrained_many_data, "constrained_many")]

merged_df = pd.concat(dfs,ignore_index=True)

merged_df.to_csv(f"{save_dir}/all_human_data_final.csv")

In [13]:
merged_df

,goal,plan,id,goal_type
0,Goal: Bake a cake.,"""Figure out what kind of cake to bake. Buy sup...",human,unconstrained
1,Goal: Bake a cake.,"""I'd get a bowl, put my butter, sugar and flou...",human,unconstrained
2,Goal: Bake a cake.,"""I would first find a recipe for the cake I wo...",human,unconstrained
3,Goal: Bake a cake.,"""Decide on what type of cake to bake. Review s...",human,unconstrained
4,Goal: Bake a cake.,"""Preheat the oven and get supplies. Next mix e...",human,unconstrained
...,...,...,...,...
835,"Goal: Take your dog for a walk, without using ...","""You will be taking your dog for a walk inside...",human,constrained_many
836,"Goal: Take your dog for a walk, without using ...","""Just take off walking with your dog in your a...",human,constrained_many
837,"Goal: Take your dog for a walk, without using ...","""Take your dog to a place where they can safel...",human,constrained_many
838,"Goal: Take your dog for a walk, without using ...","""Take the dog for a walk in the back yard. If ...",human,constrained_many
